In [0]:
!pip install -q -U watermark

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
import sklearn
import matplotlib

Using TensorFlow backend.


In [0]:
%reload_ext watermark
%watermark -a "Christian Dayan Arcos Gordillo" --iversions

tensorflow 2.2.0-rc3
pandas     1.0.3
sklearn    0.22.2.post1
numpy      1.18.3
matplotlib 3.2.1
keras      2.3.1
Christian Dayan Arcos Gordillo


In [0]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os

#Construyendo vectores de caracteristicas Features

Las redes neuronales exigen que las entradas tengan un número fijo de columnas compuestas por diferentes registros (datos). Esto exige representar los datos de forma que la NN pueda ser entrenada a partir de ellos.

Las siguientes son los tipos mas comunes de transformaciones de datos:

***Datos de Tipo Caracter (strings)***

**Nominal** - Itens discretos individuales, sin orden. Por ejemplo: color, código postal, forma.

**Ordinal** - Itens discretos individuales que pueden ser ordenados. Por ejemplo: nivel de clasificación, título del trabajo, tamaño de ropa (pequeño, mediano, grande)
***Datos Numéricos***

**Intervalo** - Valores numéricos, inicio no definido. Por ejemplo, la temperatura. nunca diria "ayer estuve dos vezes con mas fiebre que hoy".

**Ratio** - Valores numéricos, início claramente definido. Por ejemplo, la velocidad. Ud diria que "El primer carro está yendo dos vezes mas rapido que el segundo".

##Las siguientes funciones ayudan a transformar este tipo de datos

**encode_text_dummy** - Codifica los campos de texto, como las especies de de flor del dataset íris como un único campo para cada clase. Tres clases se transformarian en "0,0,1" "0,1,0" e "1,0,0". Podemos hazer el encoding de los predictores no-objetivo de esta  forma. Excelente para datos nominales.

**encode_text_index** - Codifica los campos de texto, como un único campo numérico como "0" "1" e "2". Excelente para datos nominales.

**encode_numeric_zscore** - Codifica los valores numéricos con un escore z. Las redes neuronales lidian bien con los campos "centrados", el z-score generalmente es un buen punto de partida para el intervalo / proporción.

Los valores ordinales pueden ser codificados con las funciones dummy o index. 

##Tratamiento de datos missing o NAN:

**missing_median**- Remplaza todos los valores faltantes conelo valor médio.

**to_xy** - Una vez que todos los campos son numéricos, esta función puede entregar las matrizes x e y que son usadas para entrenar la rede neuronal.

##Otras funciones de utilidad:

**hms_string** - Imprime una secuencia de tiempo recorrido.
**chart_regression** - Muestra un gráfico que dira que tan bien una regresión se esta ejecutando.

In [0]:
# Encoding los valores de texto para variables nominales
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encoding de los valores de texto para una única variable dummy. Las nuevas columnas (que no substituyen la antigua) tendran 1
# en todos los lugares donde la columna original (nombre) corresponde a cada uno de los valores-target. Una columna es adicionada para
# cada valor target.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encoding de los valores de texto para índices (osea, [1], [2], [3] para tojo, verde, azul por ejemplo).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Normalización Z-score
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convierte todos los valores faltantes en la columna especificada para la mediana
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convierte todos los valores faltantes en la columna especificada para el valor patron
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convierte un dataframe Pandas para las entradas x, y que el TensorFlow necesita
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    
    # Descubre el tipo da columna de destino. 
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    
    # Encoding para int. TensorFlow prefiere 32 bits.
    if target_type in (np.int64, np.int32):
        # Classificación
        dummies = pd.get_dummies(df[target])
        return df.values.astype('float32'), dummies.values.astype('float32')
    else:
        # Regresión
        return df.values.astype('float32'), df[target].astype('float32')

# String de tiempo bien formateado
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Gráfica de Regresión
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace = True)
    a = plt.plot(t['y'].tolist(),label = 'expected')
    b = plt.plot(t['pred'].tolist(),label = 'prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Elimina todas las filas donde la columna especificada en +/- desvios estandar
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace = True)


# Normalización de  Rango
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1, data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) * (normalized_high - normalized_low) + normalized_low

##Funciones de Visualización

Existen 3 tipos diferentes de graficos que pueden ser usados para evaluar el resultado de modelos de redes neuranales para clasificación y regresión:

* **Confusion Matrix** - Para qualquer tipo de rede neural de clasificación.
* **ROC Curve** - Para  clasificación binaria.
* **Lift Curve** - Para redes neuronales de regressión.



In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Plot de una Confusion Matrix
# Cm es la matriz de confusión, los nombres son los nombres de las clases
def plot_confusion_matrix(cm, names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(names))
    plt.xticks(tick_marks, names, rotation=45)
    plt.yticks(tick_marks, names)
    plt.tight_layout()
    plt.ylabel('Label Observado')
    plt.xlabel('Label Predicho')
    

# Plot de una curva ROC. pred (predicciones), y (valor observado)
def plot_roc(pred,y):
    fpr, tpr, _ = roc_curve(y, pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label = 'Curva ROC (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taza de Falso Positivo')
    plt.ylabel('Taza de Verdadero Positivo')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()